# Differential Equations

- https://nextjournal.com/sosiris-de

In mathematics, a differential equation is an equation that relates one or more unknown functions and their derivatives. [wikipedia](https://en.wikipedia.org/wiki/Differential_equation)

## Ordinary Differential Equation (ODE)

We define an ordinary differential equation as an equation which describes the way that a variable u changes, that is

$$
u' = f(u, p, t)
$$

where $p$ are the parameters of the model, $t$ is the time variable, and $f$ is the nonlinear model of how $u$ changes. The initial value problem also includes the information about the starting value:

$$
u(t_0) = u_0
$$

Together, if you know the starting value and you know how the value will change with time, then you know what the value will be at any time point in the future. This is the intuitive definition of a differential equation.

For Python, check [scipy.integrate.solve_ivp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html).

For Julia, check [DifferentialEquations.jl](https://docs.sciml.ai/DiffEqDocs)

## Partial Differential Equation (PDE)

A partial differential equation is a differential equation which has partial derivatives. 

**The best way to solve a PDE is...**

**By converting it into another problem!**

Generally, PDEs are converted into:

- Linear systems: `Ax = b`, find `x`.
- Nonlinear systems: `G(x) = 0`, find `x`.
- ODEs: `u' = f(u,p,t)`, find `u`.

There are thus 4 types of packages in the PDE solver pipeline:
1. Packages with ways to represent functions as vectors of numbers and their derivatives as matrices
    - There are four main ways:
        - Finite difference method (FDM): functions are represented on a grid.
        - Finite volume method (FVM): functions are represented by a discretization of its integral. 
        - Finite element method (FEM): functions are represented by a local basis. 
        - Spectral methods: functions are represented by a global basis.
2. Packages which solve linear systems
3. Packages which solve nonlinear rootfinding problems
4. Packages which solve ODEs

You need discretization tooling, linear solvers, nonlinear solvers, and finally ODE solvers to build an efficient PDE solver.

## Neural Differential Equations

- https://arxiv.org/abs/1806.07366
- https://arxiv.org/abs/2001.04385
- https://arxiv.org/abs/2202.02435

### Neural DE

The conjoining of dynamical systems and deep learning has become a topic of great interest. In particular, neural differential equations (NDEs)
demonstrate that **neural networks and differential equation are two sides of the same coin**.

A neural differential equation is a differential equation using a neural network to parameterise the vector field. The canonical example is a **neural ordinary differential equation**:

$$
y(0) = y_0 \qquad \frac{\mathrm{d} y}{\mathrm{d} t}(t) = f_\theta(t, y(t)).
$$

As a simple example, suppose we observe some picture $y_0 \in \mathbb{R}^{3 \times 32 \times 32}$ (RGB and $32 \times 32$ pixels), and wish to classify it as a picture of a cat or as a picture of a dog.

We proceed by taking $y(0) = y_0$ as the initial condition of the neural ODE, and evolve the ODE until some time $T$. An affine transformation $\ell_\theta \colon \mathbb{R}^{3 \times 32 \times 32} \to \mathbb{R}^2$ is then applied, followed by a softmax, so that the output may be interpreted as a length-2 tuple $(P(\text{picture is of a cat}), P(\text{picture is of a dog}))$.

In conventional mathematical notation, this computation may be denoted
$$
	\text{softmax}\left(\ell_\theta\left(y(0) + \int_0^T f_\theta(t, y(t)) \,\mathrm{d} t\right)\right).
$$

The parameters of the model are $\theta$. The computation graph may be backpropagated through and trained via stochastic gradient descent in the usual way. 

### Neural ODEs are the continuous limit of residual networks

Recall the formulation of a residual network:
$$
	y_{j + 1} = y_j + f_\theta(j, y_j),
$$
where $f_\theta(j, \,\cdot\,)$ is the $j$-th residual block. (The parameters of all blocks are concatenated together into $\theta$.)

Now recall the neural ODE
$$
	\frac{\mathrm{d} y}{\mathrm{d} t}(t) = f_\theta(t, y(t)).
$$
Discretising this via the explicit Euler method at times $t_j$ uniformly separated by $\Delta t$ gives
$$
	\frac{y(t_{j + 1}) - y(t_j)}{\Delta t} \approx \frac{\mathrm{d} y}{\mathrm{d} t}(t_j) = f_\theta(t_j, y(t_j)),
$$
so that
$$
	y(t_{j + 1}) = y({t_j}) + \Delta t f_\theta(t_j, y(t_j)).
$$
Absorbing the $\Delta t$ into the $f_\theta$, we recover the formulation of a residual network.

It transpires that the key features of a GRU or an LSTM, over generic recurrent networks, are updates rules that look suspiciously like discretised differential equations. StyleGAN2 and (score based) diffusion models are simply discretised SDEs. Coupling layers in invertible neural networks turn out to be related to reversible differential equation solvers. And so on.

By coincidence (or, as the idea becomes more popular, by design) many of the most effective and popular deep learning architectures resemble differential equations. Perhaps we should not be surprised: differential equations have been the dominant modelling paradigm for centuries; they are not so easily toppled.

### An important distinction: Physics-informed neural network
There has been a line of work on obtaining numerical approximations to the solution $y$ of an ODE $\frac{\mathrm{d} y}{\mathrm{d} t} = f(t, y(t))$ by representing the solution as some neural network $y = y_\theta$.

Perhaps $f$ is known, and the model $y_\theta$ is fitted by minimising a loss function of the form 
$$
\min_\theta \frac{1}{N}\sum_{i=1}^N \left|{\frac{\mathrm{d} y_\theta}{\mathrm{d} t}(t_i) - f(t_i, y_\theta(t_i))}\right|
$$
for some points $t_i \in [0, T]$. As such each solution to the differential equation is obtained by solving an optimisation problem. This has strong overtones of collocation methods or finite element methods.

This is known as a physics-informed neural network (PINN). PINNs are effective when generalised to some PDEs, in particular nonlocal or high-dimensional PDEs, for which traditional solvers are computationally expensive. (Although in most regimes traditional solvers are still the more efficient choice.)

However, we emphasise that **this is a distinct notion to neural differential equations**. NDEs use neural networks to *specify* differential equations. PINN uses neural networks to *obtain solutions to prespecified* differential equations. This distinction is a common point of confusion, especially as the PDE equivalent of PINN is sometimes referred to as a 'neural partial differential equation'.

### Physical modelling with inductive biases

Endowing a model with any known structure of a problem is known as giving the model an **inductive bias**. 'Soft' biases through penalty terms are one common example. 'Hard' biases through explicit architectural choices are another.

Physical problems often have known structure, and so a common theme has been to build in inductive biases by hybridising neural networks into this structure.

#### Universal differential equations

Consider the Lotka-Volterra model, which is a well known approach for modelling the interaction between a predator species and a prey species:
$$
	\frac{\mathrm{d} x}{\mathrm{d} t}(t) = \alpha x(t) - \beta x(t) y(t)
$$
$$
	\frac{\mathrm{d} y}{\mathrm{d} t}(t) = -\gamma x(t) + \delta x(t) y(t)
$$
Here, $x(t) \in \mathbb{R}$ and $y(t) \in \mathbb{R}$ represent the size of the population of the prey and predator species respectively, at each time $t \in [0, T]$. The right hand side is theoretically constructed, representing interactions between these species.

This theory will not usually be perfectly accurate, however. There will be some gap between the theoretical prediction and what is observed in practice. To remedy this, and letting $f_\theta, g_\theta \colon \mathbb{R}^2 \to \mathbb{R}$ be neural networks, we may instead consider the model
$$
	\frac{\mathrm{d} x}{\mathrm{d} t}(t) = \alpha x(t) - \beta x(t) y(t) + f_\theta(x(t), y(t))
$$
$$
	\frac{\mathrm{d} y}{\mathrm{d} t}(t) = -\gamma x(t) + \delta x(t) y(t) + g_\theta(x(t), y(t))
$$
in which an existing theoretical model is augmented with a neural network correction term.

We broadly refer to this approach as a **universal differential equation**. (There is little unified terminology here. Other authors have considered essentially the same idea under other names)